# **Import Libraries**

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from TF_Object import CollaborativeFilteringLayer
from TF_Object import CollaborativeFilteringModel

# **Prepare Dataset**

In [2]:
# Load all dataset in Pandas DataFrame
tourism_df = pd.read_csv("../Dataset/new_tourism_with_id_links_3.csv")

In [3]:
tourism_df.head(3)

,Place_Id,Place_Name,Category,Description,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Img_Path_0,Img_Path_1,Img_Path_2
0,1,Monumen Nasional,"Budaya, Situs Sejarah",Monumen Nasional atau yang populer disingkat d...,Jakarta,20000,"4,6",15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,images_output\Monumen Nasional\Monumen Nasiona...,images_output\Monumen Nasional\Monumen Nasiona...,images_output\Monumen Nasional\Monumen Nasiona...
1,2,Kota Tua,Budaya,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Jakarta,0,"4,6",90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,images_output\Kota Tua\Kota Tua_0.jpg,images_output\Kota Tua\Kota Tua_1.jpg,NaN
2,3,Dunia Fantasi,"Taman Hiburan, Bahari",Dunia Fantasi atau disebut juga Dufan adalah t...,Jakarta,270000,"4,6",360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,images_output\Dunia Fantasi\Dunia Fantasi_0.jpg,images_output\Dunia Fantasi\Dunia Fantasi_1.png,images_output\Dunia Fantasi\Dunia Fantasi_2.jpg


# **Initiate Important Instance**

In [4]:
# Load Y_mean for denormalize prediction results
Y_mean = np.load('../Vector/Y_mean.npy')

# Load Collaborative Filtering Model
cofi_model = tf.keras.models.load_model('Model/collaborative_filtering.h5', custom_objects={
    'CollaborativeFilteringModel': CollaborativeFilteringModel,
    'CollaborativeFilteringLayer': CollaborativeFilteringLayer
})

# **Collaborative Filtering Recommendation**

In [5]:
# Make a prediction using trained weights and biases
def get_recommendations(cofi_model, user_id):
  # Prepare User Data
  user_id = tf.constant(user_id, dtype=tf.int32)

  # Make prediction
  my_pred = cofi_model(user_id)

  # Convert to Numpy and restore the mean
  my_pred = my_pred.numpy()
  my_pred = my_pred + Y_mean
  my_pred = tf.reshape(my_pred, (-1))

  # sort predictions
  sorted_index = np.argsort(-my_pred, axis=0).reshape(-1)

  return sorted_index, my_pred

# Print recommendations
def print_recommendations(sorted_index, my_pred, num_recom):
  for i, tourist_spots_idx in enumerate(sorted_index):
    if i == num_recom:
      break
    j = tourist_spots_idx
    print(f'Predicting rating {my_pred[j]:0.2f} for {tourism_df.loc[j, "Place_Name"]}')

In [6]:
# Print 20 recommendations for user_id = 120
user_id = 5
num_recom = 20
sorted_index, my_pred = get_recommendations(cofi_model, user_id)
print_recommendations(sorted_index, my_pred, num_recom)

Predicting rating 4.86 for Klenteng Jin De Yuan
Predicting rating 4.85 for Pasar Petak Sembilan
Predicting rating 4.84 for Desa Wisata Kelor
Predicting rating 4.84 for Taman Kunang-Kunang
Predicting rating 4.84 for Taman Flora Bratang Surabaya
Predicting rating 4.74 for Skyrink - Mall Taman Anggrek
Predicting rating 4.74 for Pelabuhan Marina
Predicting rating 4.51 for Pantai Kesirat
Predicting rating 4.51 for Taman Lansia
Predicting rating 4.48 for Grojogan Watu Purbo Bangunrejo
Predicting rating 4.43 for GPIB Immanuel Semarang (Gereja Blenduk)
Predicting rating 4.39 for Taman Kasmaran
Predicting rating 4.28 for Alive Museum Ancol
Predicting rating 4.28 for Pintoe Langit Dahromo
Predicting rating 4.26 for Kawasan Malioboro
Predicting rating 4.26 for Goa Rancang Kencono
Predicting rating 4.26 for Museum Gunung Merapi
Predicting rating 4.23 for Teras Cikapundung BBWS
Predicting rating 4.21 for Kampung Cina
Predicting rating 4.20 for Pantai Nguluran
